# Different Tool Configuration Methods

This notebook demonstrates different approaches to configuring tools for LangChain agents, from simple lists to advanced error handling.

## Key Concepts
- **Method 1**: Simple list of tools (quick setup)
- **Method 2**: ToolNode with custom configuration (production-ready)
- **Error Handling**: Making agents robust to tool failures

## Why Tool Configuration Matters
- **Without error handling**: Errors crash the agent
- **With error handling**: Errors become feedback for the model
- **Model adaptation**: Agent can retry or adjust approach based on error messages

## Prerequisites

Make sure you have the required packages installed:

```bash
pip install langchain langchain-community langchain-core langgraph pydantic
ollama pull qwen3
ollama serve
```

In [ ]:
# Import required modules
from langchain_ollama import ChatOllama
from langchain.agents import create_agent, ToolNode
import tools

## Method 1: Simple List of Tools

The simplest approach - just pass tools as a list. LangChain handles everything automatically.

In [ ]:
print("=== Method 1: Simple List of Tools ===")

# Create model instance
model = ChatOllama(model="qwen3")

# Method 1: Pass list of tools (simple approach)
agent1 = create_agent(
    model, 
    tools=[tools.web_search, tools.calculate]
)

print("✓ Agent 1 created with simple tool list")
print("  - LangChain automatically creates ToolNode")
print("  - Default error handling behavior")
print("  - Good for prototyping and simple use cases")

## Method 2: Advanced ToolNode Configuration

Create a ToolNode explicitly with custom settings for better control over tool execution.

In [ ]:
print("=== Method 2: Advanced ToolNode Configuration ===")

# Method 2: Use ToolNode with error handling (advanced approach)
tool_node = ToolNode(
    tools=[tools.web_search, tools.calculate],
    handle_tool_errors="Please check your input and try again. Error details will help you correct the issue."
)

agent2 = create_agent(model, tools=tool_node)

print("✓ Agent 2 created with advanced ToolNode")
print("  - Custom error handling messages")
print("  - Better control over tool execution")
print("  - Production-ready error recovery")

## Comparing Both Approaches

Let's test both agents with the same query to see how they behave:

In [ ]:
# Test query that uses multiple tools
test_query = "Search for Python tutorials and calculate 10 times 5"

print(f"Testing both agents with: '{test_query}'")
print("=" * 60)

# Test Agent 1 (Simple)
print("\n=== Agent 1 Results (Simple Tool List) ===")
try:
    result1 = agent1.invoke({"messages": test_query})
    print(f"✓ Response: {result1['messages'][-1].content}")
except Exception as e:
    print(f"✗ Error: {e}")

# Test Agent 2 (Advanced)
print("\n=== Agent 2 Results (Advanced ToolNode) ===")
try:
    result2 = agent2.invoke({"messages": test_query})
    print(f"✓ Response: {result2['messages'][-1].content}")
except Exception as e:
    print(f"✗ Error: {e}")

print("\n💡 Both agents work the same for valid inputs, but Agent 2 handles errors better")

## Error Handling Demonstration

Let's create a scenario that might cause tool errors to see the difference:

In [ ]:
# Create a tool that might fail for demonstration
from langchain_core.tools import tool

@tool
def unreliable_tool(input_text: str) -> str:
    """A tool that sometimes fails to demonstrate error handling.
    
    Args:
        input_text: Text to process
        
    Returns:
        Processed text or error
    """
    if "error" in input_text.lower():
        raise ValueError("This tool doesn't like the word 'error'!")
    return f"Processed: {input_text}"

print("Created unreliable_tool for error demonstration")

## Creating Agents with Error-Prone Tool

In [ ]:
# Create agents with the unreliable tool
print("=== Creating Agents with Error-Prone Tool ===")

# Simple agent (no custom error handling)
simple_agent = create_agent(
    model, 
    tools=[unreliable_tool]
)

# Advanced agent (with custom error handling)
error_handling_node = ToolNode(
    tools=[unreliable_tool],
    handle_tool_errors="The tool encountered an error. Please try a different approach or modify your request."
)

advanced_agent = create_agent(model, tools=error_handling_node)

print("✓ Both agents created with error-prone tool")

## Testing Error Scenarios

In [ ]:
print("=== Testing Error Scenarios ===")

# Test 1: Normal operation (should work for both)
normal_query = "Process this text: Hello World"
print(f"\n1. Normal query: '{normal_query}'")

try:
    result = simple_agent.invoke({"messages": normal_query})
    print(f"   Simple agent: ✓ Success")
except Exception as e:
    print(f"   Simple agent: ✗ Failed - {e}")

try:
    result = advanced_agent.invoke({"messages": normal_query})
    print(f"   Advanced agent: ✓ Success")
except Exception as e:
    print(f"   Advanced agent: ✗ Failed - {e}")

# Test 2: Error-inducing query
error_query = "Process this error message"
print(f"\n2. Error-inducing query: '{error_query}'")

try:
    result = simple_agent.invoke({"messages": error_query})
    print(f"   Simple agent: ✓ Handled gracefully")
except Exception as e:
    print(f"   Simple agent: ✗ Failed - {type(e).__name__}")

try:
    result = advanced_agent.invoke({"messages": error_query})
    print(f"   Advanced agent: ✓ Handled gracefully")
    print(f"   Response: {result['messages'][-1].content[:100]}...")
except Exception as e:
    print(f"   Advanced agent: ✗ Failed - {type(e).__name__}")


## Advanced ToolNode Features

The ToolNode class offers many configuration options:

In [ ]:
print("=== Advanced ToolNode Configuration Options ===")

# Example of more advanced ToolNode configuration
advanced_tool_node = ToolNode(
    tools=[tools.web_search, tools.calculate, tools.analyze_text],
    
    # Custom error message
    handle_tool_errors=(
        "I encountered an issue with that tool. "
        "Let me try a different approach or you can rephrase your request."
    ),
    
    # Additional configuration could include:
    # - Custom tool selection logic
    # - Tool execution timeouts
    # - Result validation
    # - Logging and monitoring
)

production_agent = create_agent(model, tools=advanced_tool_node)

print("✓ Production-ready agent created with:")
print("  - Custom error messages")
print("  - Multiple tools configured")
print("  - Robust error handling")
print("  - Ready for production deployment")

## Tool Organization Strategies

For larger applications, consider organizing tools by category:

In [ ]:
print("=== Tool Organization Strategies ===")

# Organize tools by category
search_tools = [tools.web_search, tools.latest_news]
analysis_tools = [tools.analyze_data, tools.analyze_text]
utility_tools = [tools.calculate, tools.extract_contact]
preference_tools = [tools.save_user_preference, tools.get_user_preference]

# Create specialized agents for different purposes
search_agent = create_agent(
    model,
    tools=ToolNode(
        tools=search_tools,
        handle_tool_errors="Search functionality encountered an issue. Let me try alternative search methods."
    )
)

analysis_agent = create_agent(
    model,
    tools=ToolNode(
        tools=analysis_tools,
        handle_tool_errors="Analysis tools encountered an issue. Please check your data format and try again."
    )
)

# General purpose agent with all tools
general_agent = create_agent(
    model,
    tools=ToolNode(
        tools=search_tools + analysis_tools + utility_tools + preference_tools,
        handle_tool_errors="I encountered an issue with that operation. Let me try a different approach."
    )
)

print("✓ Created specialized agents:")
print(f"  - Search agent: {len(search_tools)} tools")
print(f"  - Analysis agent: {len(analysis_tools)} tools")
print(f"  - General agent: {len(search_tools + analysis_tools + utility_tools + preference_tools)} tools")

## Testing Specialized Agents

In [ ]:
print("=== Testing Specialized Agents ===")

# Test search agent
print("\n1. Testing Search Agent:")
search_query = "Find information about machine learning"
try:
    search_result = search_agent.invoke({"messages": search_query})
    print("   ✓ Search agent completed successfully")
except Exception as e:
    print(f"   ✗ Search agent failed: {e}")

# Test analysis agent
print("\n2. Testing Analysis Agent:")
analysis_query = "Analyze this text: The quick brown fox jumps over the lazy dog"
try:
    analysis_result = analysis_agent.invoke({"messages": analysis_query})
    print("   ✓ Analysis agent completed successfully")
except Exception as e:
    print(f"   ✗ Analysis agent failed: {e}")

# Test general agent with mixed query
print("\n3. Testing General Agent:")
mixed_query = "Search for Python tutorials, then calculate 15 * 23, and analyze the text 'Hello World'"
try:
    general_result = general_agent.invoke({"messages": mixed_query})
    print("   ✓ General agent completed successfully")
except Exception as e:
    print(f"   ✗ General agent failed: {e}")

## Best Practices Summary

### When to Use Simple Tool Lists
- **Prototyping and experimentation**
- **Simple applications** with basic tool needs
- **Learning and tutorials**
- **Quick demos** and proof-of-concepts

### When to Use Advanced ToolNode
- **Production applications**
- **Error-prone environments**
- **Complex tool interactions**
- **Applications requiring robustness**
- **Custom error handling needs**

### Tool Organization Guidelines

1. **Group by Function**: Search, analysis, utility tools
2. **Separate by Domain**: Industry-specific tool sets
3. **Consider Performance**: Frequently used vs. specialized tools
4. **Plan for Scale**: Modular tool architecture

### Error Handling Strategies

1. **Graceful Degradation**: Continue operation when possible
2. **Informative Messages**: Help users understand what went wrong
3. **Retry Logic**: Allow agents to attempt alternative approaches
4. **Logging**: Track errors for debugging and improvement

### Configuration Tips

1. **Start Simple**: Begin with basic tool lists
2. **Add Complexity Gradually**: Implement advanced features as needed
3. **Test Error Scenarios**: Ensure robust behavior under failure
4. **Monitor in Production**: Track tool usage and errors
5. **Document Tool Behavior**: Clear descriptions for maintenance

## Conclusion

Tool configuration is crucial for building reliable agents:
- **Simple lists** work great for getting started
- **ToolNode** provides production-ready robustness
- **Error handling** prevents agent crashes
- **Organization** improves maintainability
- **Specialization** can improve performance